In [1]:
## Installing PySpark in Jupyter Notebook and other environments.

%pip install findspark
%pip install pyspark
%pip install py4j



Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
### Import all dependencies ### for the Spark job
# Importing necessary libraries

import findspark
import logging
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql import Row
from pyspark.sql import DataFrame
from pyspark.sql.functions import regexp_extract, col
from pyspark.sql.functions import year
from pyspark.sql.functions import col
from datetime import datetime



In [3]:
findspark.init()

In [4]:
# Set up logging
logging.basicConfig(level=logging.INFO)

# Initialize SparkSession
spark = SparkSession.builder. \
    appName("data_engineering_task"). \
    getOrCreate()

25/04/16 14:22:44 WARN Utils: Your hostname, codespaces-4e7499 resolves to a loopback address: 127.0.0.1; using 10.0.3.18 instead (on interface eth0)
25/04/16 14:22:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/16 14:22:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


#Reading data and printing schema values

In [5]:
# Load the dataset
data_file = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .option("delimiter", ",") \
    .load("/workspaces/data_engineering_task/dataset/nyc-jobs.csv")



# Remove duplicate rows
print("count before remove duplicate",data_file.count())
#data_file = data_file.dropDuplicates()
# Print the schema to understand column types
print("count after removed duplicate",data_file.count())
# Print the schema to understand column types
data_file.printSchema()
# Show the first few rows of the DataFrame
#data_file.show(5)

count before remove duplicate 2946
count after removed duplicate 2946
root
 |-- Job ID: integer (nullable = true)
 |-- Agency: string (nullable = true)
 |-- Posting Type: string (nullable = true)
 |-- # Of Positions: integer (nullable = true)
 |-- Business Title: string (nullable = true)
 |-- Civil Service Title: string (nullable = true)
 |-- Title Code No: string (nullable = true)
 |-- Level: string (nullable = true)
 |-- Job Category: string (nullable = true)
 |-- Full-Time/Part-Time indicator: string (nullable = true)
 |-- Salary Range From: double (nullable = true)
 |-- Salary Range To: double (nullable = true)
 |-- Salary Frequency: string (nullable = true)
 |-- Work Location: string (nullable = true)
 |-- Division/Work Unit: string (nullable = true)
 |-- Job Description: string (nullable = true)
 |-- Minimum Qual Requirements: string (nullable = true)
 |-- Preferred Skills: string (nullable = true)
 |-- Additional Information: string (nullable = true)
 |-- To Apply: string (nulla

In [6]:

# Summary statistics for numerical columns
data_file.describe(["# Of Positions", "Salary Range From", "Salary Range To"]).show()

# Count distinct values for categorical columns
categorical_columns = ["Agency", "Posting Type", "Title Code No", "Level", "Job Category", "Full-Time/Part-Time indicator", "Salary Frequency"]

for column in categorical_columns:
    print(f"Distinct values in {column}:")
    data_file.select(column).distinct().show(10, truncate=False)


# Count null values in each column
data_file.select([sum(col(c).isNull().cast("int")).alias(c) for c in data_file.columns]).show()

+-------+------------------+------------------+-----------------+
|summary|    # Of Positions| Salary Range From|  Salary Range To|
+-------+------------------+------------------+-----------------+
|  count|              2946|              2946|             2946|
|   mean|2.4959266802443993| 58904.13979385609|85535.71162739306|
| stddev| 9.281312826466838|26986.575935791363|42871.31345366745|
|    min|                 1|               0.0|            10.36|
|    max|               200|          218587.0|         234402.0|
+-------+------------------+------------------+-----------------+

Distinct values in Agency:
+------------------------------+
|Agency                        |
+------------------------------+
|OFFICE OF COLLECTIVE BARGAININ|
|FIRE DEPARTMENT               |
|ADMIN FOR CHILDREN'S SVCS     |
|MANHATTAN COMMUNITY BOARD #8  |
|TAXI & LIMOUSINE COMMISSION   |
|DEPARTMENT OF BUSINESS SERV.  |
|DEPT OF DESIGN & CONSTRUCTION |
|FINANCIAL INFO SVCS AGENCY    |
|DEPARTMENT OF 

25/04/16 14:22:55 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+------+------+------------+--------------+--------------+-------------------+-------------+-----+------------+-----------------------------+-----------------+---------------+----------------+-------------+------------------+---------------+-------------------------+----------------+----------------------+--------+-----------+---------------+-------------------+---------------------+------------+----------+---------------+------------+
|Job ID|Agency|Posting Type|# Of Positions|Business Title|Civil Service Title|Title Code No|Level|Job Category|Full-Time/Part-Time indicator|Salary Range From|Salary Range To|Salary Frequency|Work Location|Division/Work Unit|Job Description|Minimum Qual Requirements|Preferred Skills|Additional Information|To Apply|Hours/Shift|Work Location 1|Recruitment Contact|Residency Requirement|Posting Date|Post Until|Posting Updated|Process Date|
+------+------+------------+--------------+--------------+-------------------+-------------+-----+------------+---------

#List of KPIs to be resolved

In [7]:
# 1. Count the number of jobs per agency
agency_counts = data_file.groupBy("Agency").count().orderBy("count", ascending=False)
agency_counts.show(10, truncate=False)

+------------------------------+-----+
|Agency                        |count|
+------------------------------+-----+
|DEPT OF ENVIRONMENT PROTECTION|655  |
|NYC HOUSING AUTHORITY         |231  |
|DEPT OF HEALTH/MENTAL HYGIENE |188  |
|DEPARTMENT OF TRANSPORTATION  |183  |
|DEPT OF DESIGN & CONSTRUCTION |142  |
|TAXI & LIMOUSINE COMMISSION   |134  |
|ADMIN FOR CHILDREN'S SVCS     |108  |
|DEPT OF INFO TECH & TELECOMM  |107  |
|LAW DEPARTMENT                |95   |
|HOUSING PRESERVATION & DVLPMNT|86   |
+------------------------------+-----+
only showing top 10 rows



In [8]:
# 2.number of jobs posting per category top 10:
job_category_counts = data_file.filter(col("Job Category").isNotNull()).groupBy("Job Category").count().orderBy("count", ascending=False)
job_category_counts.show(10, truncate=False)

+-----------------------------------------+-----+
|Job Category                             |count|
+-----------------------------------------+-----+
|Engineering, Architecture, & Planning    |504  |
|Technology, Data & Innovation            |313  |
|Legal Affairs                            |226  |
|Public Safety, Inspections, & Enforcement|182  |
|Building Operations & Maintenance        |181  |
|Finance, Accounting, & Procurement       |169  |
|Administration & Human Resources         |134  |
|Constituent Services & Community Programs|129  |
|Health                                   |125  |
|Policy, Research & Analysis              |124  |
+-----------------------------------------+-----+
only showing top 10 rows



In [9]:
# 3. salary distribution per job category
salary_distribution = data_file.groupBy("Job Category").agg(sum("Salary Range From").alias("Total Salary Range From"),
                                                            sum("Salary Range To").alias("Total Salary Range To"),
                                                            sum("# Of Positions").alias("Total Positions")) \
    .orderBy("Total Salary Range From", ascending=False)
salary_distribution.show(10, truncate=False)  




+-----------------------------------------+-----------------------+---------------------+---------------+
|Job Category                             |Total Salary Range From|Total Salary Range To|Total Positions|
+-----------------------------------------+-----------------------+---------------------+---------------+
|Engineering, Architecture, & Planning    |3.3320681810000002E7   |5.188658181E7        |762            |
|Technology, Data & Innovation            |2.1777043960500002E7   |3.33524432098E7      |405            |
|Legal Affairs                            |1.58365392558E7        |2.17911463106E7      |515            |
|Finance, Accounting, & Procurement       |1.00772180668E7        |1.48655259762E7      |275            |
|Public Safety, Inspections, & Enforcement|9363633.7292           |1.32629547906E7      |1407           |
|Policy, Research & Analysis              |6804147.8412           |8963123.9008         |200            |
|Constituent Services & Community Programs|646

In [10]:
# 4.Correlation between the higher degree and the salary.

data_with_degree = data_file.withColumn(
    "Degree Type",
    regexp_extract(col("Minimum Qual Requirements"), r"(\w+)\sdegree", 1)
)

# Show the results
data_with_degree.select("Degree Type").show(10, truncate=False)

+-------------+
|Degree Type  |
+-------------+
|baccalaureate|
|baccalaureate|
|             |
|             |
|             |
|             |
|baccalaureate|
|baccalaureate|
|s            |
|             |
+-------------+
only showing top 10 rows



In [11]:
# 5. Job posting having the highest salary per agency
highest_salary_per_agency = data_file.groupBy("Agency").agg(
    {"Salary Range From": "max", "Salary Range To": "max"}
).orderBy("max(Salary Range To)", ascending=False)
highest_salary_per_agency.show(10, truncate=False)

+------------------------------+----------------------+--------------------+
|Agency                        |max(Salary Range From)|max(Salary Range To)|
+------------------------------+----------------------+--------------------+
|NYC EMPLOYEES RETIREMENT SYS  |130000.0              |234402.0            |
|POLICE DEPARTMENT             |200000.0              |234402.0            |
|NYC HOUSING AUTHORITY         |175000.0              |234402.0            |
|DEPT OF HEALTH/MENTAL HYGIENE |157725.0              |225217.0            |
|DEPT OF ENVIRONMENT PROTECTION|218587.0              |218587.0            |
|DEPT OF DESIGN & CONSTRUCTION |86346.0               |217244.0            |
|DISTRICT ATTORNEY KINGS COUNTY|175000.0              |208826.0            |
|DEPARTMENT OF SANITATION      |87490.0               |202744.0            |
|DEPT OF INFO TECH & TELECOMM  |96020.0               |189000.0            |
|DEPARTMENT OF PROBATION       |79620.0               |180000.0            |

25/04/16 14:22:59 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [ ]:
# 6. Job positings average salary per agency for the last 2 years

#finding most lasted year to find last 2 years data.
 
find_year = data_file.withColumn("Year", year(col("Posting Date"))).distinct()
current_year=find_year.filter(col("year").isNotNull()).select("Year").orderBy("Year",ascending=False).first()["Year"]


# Filter data for the last 2 years
filtered_data = data_file.filter(
    (col("Posting Date").isNotNull()) & 
    (year(col("Posting Date")) >= (current_year - 2))
)

# Calculate average salary per agency
average_salary_per_agency = filtered_data.filter(col("Posting Date").isNotNull()) \
    .groupBy("Agency","Posting Date") \
    .agg({"Salary Range From": "avg", "Salary Range To": "avg"}) \
    .orderBy("avg(Salary Range To)", ascending=False)
average_salary_per_agency.show(10, truncate=False)

+------------------------------+-----------------------+----------------------+--------------------+
|Agency                        |Posting Date           |avg(Salary Range From)|avg(Salary Range To)|
+------------------------------+-----------------------+----------------------+--------------------+
|NYC HOUSING AUTHORITY         |2019-05-14T00:00:00.000|103620.0              |234402.0            |
|POLICE DEPARTMENT             |2019-10-30T00:00:00.000|200000.0              |234402.0            |
|DEPT OF ENVIRONMENT PROTECTION|2019-05-28T00:00:00.000|218587.0              |218587.0            |
|DEPT OF DESIGN & CONSTRUCTION |2019-10-11T00:00:00.000|86346.0               |217244.0            |
|NYC HOUSING AUTHORITY         |2019-01-29T00:00:00.000|78574.0               |202744.0            |
|DEPT OF ENVIRONMENT PROTECTION|2017-03-02T00:00:00.000|75338.0               |194395.0            |
|DEPT OF ENVIRONMENT PROTECTION|2017-04-06T00:00:00.000|75338.0               |194395.0    

In [ ]:
# 7 highest paid skills in the US market



### Sample function

In [ ]:
#highest paid skills in the US market


In [ ]:
def get_salary_frequency(df: DataFrame) -> list:
    row_list = df.select('Salary Frequency').distinct().collect()
    return [row['Salary Frequency'] for row in row_list]

### Example of test function

In [ ]:
mock_data = [('A', 'Annual'), ('B', 'Daily')]
expected_result = ['Annual', 'Daily']

In [ ]:
def test_get_salary_frequency(mock_data: list, 
                              expected_result: list,
                              schema: list = ['id', 'Salary Frequency']):  
    mock_df = spark.createDataFrame(data = mock_data, schema = schema)
    assert get_salary_frequency(mock_df) == expected_result